# Setup

In [1]:
import json
import requests
from datetime import datetime as dt
from collections import defaultdict
import pandas as pd

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display, Markdown

In [3]:
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [4]:
def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    return response.json()

def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/1.1/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def aragorn(message, coalesce_type='xnone'):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/1.1/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/1.1/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

##

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)

def geneticskp(message):
    url = 'https://translator.broadinstitute.org/genetics_provider/trapi/v1.1/query'
    return post(strider,url,message)

def coalesce(message,method='all'):
    url = 'https://answercoalesce.renci.org/coalesce/graph'
    return post('AC'+method,url,message)

def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/all',strider_answer)
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',coalesced_answer)
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',omni_answer)
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',weighted_answer)
    return strider_answer,coalesced_answer,omni_answer,weighted_answer,scored_answer

In [5]:
def print_errors(strider_result):
    errorcounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if logmessage['level'] == 'ERROR':
            jm = json.loads(logmessage['message'])
            words = jm['error'].split()
            e = " ".join(words[:-5])
            errorcounts[e] += 1
    for error,count in errorcounts.items():
        print(f'{error} ({count} times)')
        
def print_queried_sources(strider_result):
    querycounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if 'step' in logmessage and isinstance(logmessage['step'],list):
            for s in logmessage['step']:
                querycounts[s['url']] += 1
    for url,count in querycounts.items():
        print(f'{url} ({count} times)')
        
def print_query_for_source(strider_result,url):
    for logmessage in strider_result['logs']:
        if 'step' in logmessage and isinstance(logmessage['step'],list):
            for s in logmessage['step']:
                if s['url']==url:
                    print(s)

In [6]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    results = {}
    for child in j['children']:
        if child['actor']['agent'] in ['ara-aragorn', 'ara-aragorn-exp']:
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
            print( child['status'], child['actor']['agent'],nresults )
    return results

In [7]:
def get_provenance(message):
    """Given a message with results, find the source of the edges"""
    prov = defaultdict(lambda: defaultdict(int)) # {qedge->{source->count}}
    results = message['message']['results']
    kg = message['message']['knowledge_graph']['edges']
    edge_bindings = [ r['edge_bindings'] for r in results ]
    for bindings in edge_bindings:
        for qg_e, kg_l in bindings.items():
            for kg_e in kg_l:
                for att in kg[kg_e['id']]['attributes']:
                    if att['attribute_type_id'] == 'MetaInformation:Provenance':
                        source = att['value']
                        prov[qg_e][source]+=1
    qg_edges = []
    sources = []
    counts = []
    for qg_e in prov:
        for source in prov[qg_e]:
            qg_edges.append(qg_e)
            sources.append(source)
            counts.append(prov[qg_e][source])
    prov_table = pd.DataFrame({"QG Edge":qg_edges, "Source":sources, "Count":counts})
    return prov_table

## Query Specific

In [8]:
standup_json='StandupDefinitions/standup_20.json'

In [9]:
with open(standup_json,'r') as jsonfile:
    standup_info = json.load(jsonfile)

In [10]:
display(Markdown(f"# {standup_info['Query Title']}"))
display(Markdown(f"{standup_info['Query Description']}"))
print(f'Github Issue: {standup_info["github_issue"]}')

# 20. Script 1c

Phenotypic Feature - Gene KCNMA1 (HGNC:6284)

Github Issue: https://github.com/NCATSTranslator/testing/issues/70


The query as run through the ARS:

In [11]:
query = json.loads(requests.get(standup_info['query_location']).content)
printjson(query)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "categories": [
                        "biolink:PhenotypicFeature"
                    ]
                },
                "n1": {
                    "ids": [
                        "HGNC:6284"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1"
                }
            }
        }
    }
}


## ARS Assessment

In [12]:
ARS_Responses = [(dt.strptime(x['ARS_result_date'],'%Y-%m-%d'),x['ARS_result_id']) for x in standup_info['ARS_Results']]
ARS_Responses.sort()

In [14]:
for ars_date, ars_id in ARS_Responses:
    display(Markdown(f'### {ars_date}'))
    #_ = retrieve_ars_results(ars_id)
    print(f'https://arax.ncats.io/?source=ARS&id={ars_id}')

### 2021-06-01 00:00:00

https://arax.ncats.io/?source=ARS&id=21fca295-6746-47f9-ab31-c79f3f9dc398


## Strider Direct

In [16]:
start = dt.now()
strider_result = strider(query)
end = dt.now()
print(f"Strider produced {len(strider_result['message']['results'])} results in {end-start}.")

Strider produced 6 results in 0:00:11.164515.


In [17]:
from collections import defaultdict
dd = defaultdict(int)
for r in strider_result['message']['results']:
    eb = r['edge_bindings']['e0'][0]['id']
    edge = strider_result['message']['knowledge_graph']['edges'][eb]
    k = (r['node_bindings']['n0'][0]['id'],  edge['predicate'], edge['attributes'][0]['value'])
    dd[k] += 1
    #print(r['node_bindings']['n0'][0]['id'], eb, edge['predicate'], edge['attributes'][0]['value'])
#for k,v in dd.items():
#    print(k,v)
res = list(dd.keys())
res.sort()
for r in res:
    print(r, dd[r])

('HP:0002187', 'biolink:gene_associated_with_condition', 'https://automat.renci.org/pharos/1.1/query') 1
('HP:0003811', 'biolink:gene_associated_with_condition', 'https://automat.renci.org/robokopkg/1.1/query') 1
('HP:0004324', 'biolink:gene_associated_with_condition', 'https://automat.renci.org/robokopkg/1.1/query') 1
('HP:0010849', 'biolink:gene_associated_with_condition', 'https://automat.renci.org/robokopkg/1.1/query') 1
('HP:0200134', 'biolink:gene_associated_with_condition', 'https://automat.renci.org/pharos/1.1/query') 1
('MONDO:0016370', 'biolink:gene_associated_with_condition', 'https://automat.renci.org/robokopkg/1.1/query') 1


### Provenance

In [19]:
prov = get_provenance(strider_result)
display(prov)

,QG Edge,Source,Count
0,e0,https://automat.renci.org/pharos/1.1/query,5
1,e0,https://automat.renci.org/robokopkg/1.1/query,4


### Queried sources

In [20]:
print_queried_sources(strider_result)

### Errors

In [21]:
print_errors(strider_result)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Results

In [22]:
view = GammaViewer(props={"data":strider_result})
display(view)

### Strider Assessment

Enter Assessment Here

In [23]:
start = dt.now()
gkp_result = geneticskp(query)
end = dt.now()
print(f"GKP produced {len(gkp_result['message']['results'])} results in {end-start}.")

GKP produced 3 results in 0:00:00.367179.


In [ ]:
printjson(gkp_result['message']['results'])

In [ ]:
query10 = {
    "message": {
        "query_graph": {
            "edges": {
                "e01": {
                    "object": "n0",
                    "subject": "n1",
                    "predicate": 
                        "biolink:related_to"
                }
            },
            "nodes": {
                "n0": {
                    "id": 
                        "NCBIGene:23221",
                    "category": 
                        "biolink:Gene"
                },
                "n1": {
                    "category":
                        "biolink:Gene"
                }
            }
        }
    }
}

In [ ]:
start = dt.now()
gkp_result = geneticskp(query10)
end = dt.now()
print(f"GKP produced {len(gkp_result['message']['results'])} results in {end-start}.")

## ARAGORN 

In [ ]:
start = dt.now()
aragorn_result = aragorn(query)
end = dt.now()
if 'results' in aragorn_result['message']:
    print(f"ARAGORN produced {len(aragorn_result['message']['results'])} results in {end-start}.")
else:
    print('Error, no result field')

In [ ]:
print_json(aragorn_result['message'])

In [ ]:
view = GammaViewer(props={"data":aragorn_result})
display(view)

In [ ]:
tq={
    "message": {
        "query_graph": {
            "nodes": {
                "n1": {
                    "id": "HP:0033127",
                    "category": "biolink:PhenotypicFeature"
                },
                "n2": {
                    "category": "biolink:Disease"
                }
            },
            "edges": {
                "e02": {
                    "subject": "n1",
                    "object": "n2",
                    "predicate": "biolink:affected_by"
                }
            }
        }
    }
}

In [ ]:
r = automat('uberongraph',tq)

### ARAGORN Assessment

How did we do?

In [ ]:
query = requests.get('https://kp-registry.renci.org/kps').json()
printjson(query)